# Data Checks Introduction

Miscellaneous, slightly hacky checks on the raw events to help diagnose data warehouse issues




# Settings

In [142]:
from datetime import datetime, timedelta

In [143]:
num_days_to_query = 3
#from_datetime = datetime.now() - timedelta(days = 5)
#from_datetime = datetime(year=2020, month=1, day=4)
#to_datetime = from_datetime+ timedelta(days=num_days_to_query)
to_datetime = datetime(year=2020, month=3, day=1)
from_datetime = to_datetime - timedelta(days=num_days_to_query)
include_device_segmentation = True #E.g. iphone users.  This will use more memory (and likely slow things a bit).

# Imports

In [144]:
# Run imports that might require installation to the environment, and install if necessary.
try:
    import psycopg2
except:
    print("Failed ot import psychopg2, trying to install it")
    !{sys.executable} -m pip install psycopg2-binary
    import psycopg2
    print("Successfully installed")
    
    
try:
    import dateparser
except:
    print("Failed ot import dateparser, trying to install it")
    #!{sys.executable} -m pip install dateparser
    !pip install dateparser
    import dateparser
    print("Successfully installed")
    
try:
    import pyathena #used in other imports, so really just checking it's available
except:
    print("Failed ot import pyathena, trying to install it")
    ! pip install pyathena
    #!{sys.executable} -m pip install pyathena
    import pyathena
    print("Successfully installed")
    
try:
    import user_agents
except:
    print("Failed ot import user_agents, trying to install it")
    #!{sys.executable} -m pip install user_agents
    !pip install user_agents
    import user_agents
    print("Successfully installed")

    
# Imports on files that might have dependencies that need installing

from athena_querying import AthenaQuery
from athena_common_queries import *


import ipywidgets as widgets
    

# Collect the Raw Events

In [145]:
aq = AthenaQuery()
aq.connect()

In [146]:
query = create_generic_event_query(from_datetime, to_datetime, include_user_agent=include_device_segmentation, include_ip_address = include_device_segmentation, interpret_urls=False)


In [147]:
events = aq.query(query)

/home/ec2-user/anaconda3/envs/python3/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2963: DtypeWarning: Columns (7) have mixed types. Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [148]:
events.head()

,sent_at_timestamp,sent_at,date,type,event_name,status,anonymous_id,amp_id,page_url,referrer,user_agent,ip_address
0,2020-02-28 10:05:53.144,2020-02-28T10:05:53.144Z,2020-02-28,event,Reading,Article Body 25,52640dfe-999d-478e-837e-3303570a1474,NaN,https://blog.moneysmart.sg/budgeting/delivery-...,https://www.google.com/,Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:7...,101.78.121.215
1,2020-02-28 10:04:45.189,2020-02-28T10:04:45.189Z,2020-02-28,page,PageView,NaN,81a88c2c-a0d3-4671-9ecc-6027f7324d07,NaN,https://blog.moneysmart.sg/fixed-deposits/best...,https://www.google.com.sg/,Mozilla/5.0 (iPhone; CPU iPhone OS 13_3_1 like...,119.74.192.149
2,2020-02-28 10:04:45.208,2020-02-28T10:04:45.208Z,2020-02-28,event,Reading,Article Body 25,234c5fc6-62d5-4622-8302-13dbfa8343eb,NaN,https://www.moneysmart.tw/articles/%E5%AE%9A%E...,https://www.google.com/,Mozilla/5.0 (iPhone; CPU iPhone OS 13_3 like M...,118.233.70.27
3,2020-02-28 10:04:44.111,2020-02-28T10:04:44.111Z,2020-02-28,event,Reading,Article Body 50,57b19bb5-d2dc-4e29-99db-a7f9024e62be,NaN,https://blog.moneysmart.hk/zh-hk/private-housi...,https://www.google.com/,Mozilla/5.0 (Linux; Android 10; Pixel 4) Apple...,223.17.1.84
4,2020-02-28 10:04:44.183,2020-02-28T10:04:44.183Z,2020-02-28,event,Reading,Article Body 75,2212de22-00a3-4eb8-bf77-f21841df79f4,NaN,https://blog.moneysmart.sg/invest/saxo-capital...,https://www.google.com/,Mozilla/5.0 (Linux; Android 9; SM-N960F) Apple...,2401:7400:6005:1f54:1:1:ea98:eb85


In [149]:
query = "select split_part(context.page_url, '?', 1) as root_url, version, count(*) as count, min(sent_at), max(sent_at) from ms_data_lake_production.ms_data_stream_production_processed where "
query += create_partition_filter(from_datetime, to_datetime)
query += " group by 1,2"

In [150]:
print(query)

select split_part(context.page_url, '?', 1) as root_url, version, count(*) as count, min(sent_at), max(sent_at) from ms_data_lake_production.ms_data_stream_production_processed where 
  (
 partition_0 >= '2020'
 AND partition_1 >= '02'
 AND partition_2 >= '27'
 OR (
 partition_0 >= '2020'
 AND partition_1 > '02'
 ) 
 OR (
 partition_0 > '2020'
 ) 
)
 AND ((partition_0 <= '2020'
	 AND partition_1 <= '03'
	 AND partition_2 <= '01'
) 
 OR (
	 partition_0 <= '2020'
	 AND partition_1 < '03'
) 
 OR (
	 partition_0 < '2020'
) 
) group by 1,2


In [151]:
df = aq.query(query)

In [152]:
df.head(10)

,root_url,version,count,_col3,_col4
0,https://blog.moneysmart.sg/budgeting/cost-livi...,2.6.0,3859,2020-02-26T16:00:52.156Z,2020-03-01T15:59:43.116Z
1,https://blog.moneysmart.sg/dining/meal-subscri...,2.6.0,619,2020-02-26T16:03:07.131Z,2020-03-01T15:30:03.312Z
2,https://get.moneysmart.sg/scb-300/,2.6.0,142,2020-02-26T16:22:54.580Z,2020-03-01T15:56:22.153Z
3,https://blog3.moneysmart.hk/zh-hk/family/%E8%8...,2.6.0,3,2020-02-26T21:29:59.328Z,2020-02-26T21:30:22.664Z
4,https://blog3.moneysmart.sg/shopping/value-dol...,2.6.0,113,2020-02-26T16:22:36.075Z,2020-03-01T12:25:17.812Z
5,https://www.moneysmart.sg/travel-insurance/msi...,2.6.0,53,2020-02-26T16:29:36.716Z,2020-03-01T14:17:57.977Z
6,https://www.moneysmart.tw/articles/costco辦卡-聯名...,3.0.9,2,2020-02-27T00:52:09.276Z,2020-02-27T20:56:25.099Z
7,https://www.moneysmart.tw/articles/%E8%A4%87%E...,3.0.9,707,2020-02-26T15:59:46.545Z,2020-03-01T15:46:32.026Z
8,https://www.moneysmart.hk/zh-hk/credit-cards/b...,2.6.0,295,2020-02-26T16:16:45.740Z,2020-03-01T15:35:52.826Z
9,https://blog3.moneysmart.sg/education/singapor...,2.6.0,15,2020-02-27T08:10:04.310Z,2020-03-01T04:10:08.827Z


In [153]:
df["root_url"]

0        https://blog.moneysmart.sg/budgeting/cost-livi...
1        https://blog.moneysmart.sg/dining/meal-subscri...
2                       https://get.moneysmart.sg/scb-300/
3        https://blog3.moneysmart.hk/zh-hk/family/%E8%8...
4        https://blog3.moneysmart.sg/shopping/value-dol...
                               ...                        
12324    https://blog.moneysmart.hk/zh-hk/investment/%e...
12325    https://blog.moneysmart.sg/property/3-common-m...
12326    https://www.moneysmart.tw/articles/全聯雙人牌廚具-集點換...
12327    https://blog3.moneysmart.hk/zh-hk/credit-cards...
12328    https://www.moneysmart.id/tingkatkan-produktiv...
Name: root_url, Length: 12329, dtype: object

In [154]:
df["root_url"] = df["root_url"].str.strip("/")
df_g = df.groupby(["root_url", "version"]).sum().reset_index()

In [155]:
df_g.head(20)

,root_url,version,count
0,file:///C:/Users/Dell/Downloads/COVID-19%20Is%...,2.4.20,4
1,file://C:\Users\Dell\Downloads\COVID-19 Is Dis...,2.4.20,1
2,file://F:\SINGAPORE JANICE\COURSES MODULES\9 A...,2.4.20,1
3,http://iss.moneysmart.sg/credit-cards/standard...,2.6.0,1
4,http://iss.moneysmart.sg/travel-insurance/msig...,2.6.0,2
5,http://iss.moneysmart.sg/travel-insurance/ntuc...,2.6.0,2
6,http://localhost:3000/car-insurance/fwd-ms,2.6.0,3
7,http://localhost:3000/credit-cards/airport-lou...,2.4.20,1
8,http://localhost:3000/credit-cards/airport-lou...,2.6.0,14
9,http://localhost:3000/zh-hk/credit-cards/coffe...,2.4.20,1


In [156]:
pv = pd.pivot_table(df_g, index=["root_url"], values=["count"], columns=["version"], fill_value=0)

In [157]:
pv[("count", "total")] = pv.sum(axis=1)

In [158]:
pv.head()

count                     \
version                                            2.4.19 2.4.20 2.5.4 2.6.0   
root_url                                                                       
file:///C:/Users/Dell/Downloads/COVID-19%20Is%2...      0      4     0     0   
file://C:\Users\Dell\Downloads\COVID-19 Is Dist...      0      1     0     0   
file://F:\SINGAPORE JANICE\COURSES MODULES\9 Af...      0      1     0     0   
http://iss.moneysmart.sg/credit-cards/standard-...      0      0     0     1   
http://iss.moneysmart.sg/travel-insurance/msig-...      0      0     0     2   

                                                                      
version                                            3.0.8 3.0.9 total  
root_url                                                              
file:///C:/Users/Dell/Downloads/COVID-19%20Is%2...     0     0     4  
file://C:\Users\Dell\Downloads\COVID-19 Is Dist...     0     0     1  
file://F:\SINGAPORE JANICE\COURSES MODULES\9 Af...     0     0     1  
http://iss.moneysmart.sg/credit-cards/standard-...     0     0     1  
http://iss.moneysmart.sg/travel-insurance/msig-...     0     0     2

In [159]:
pd.options.display.max_rows = 100

In [160]:
bad_tracker_urls = pv[(pv[("count","2.6.0")]==0)  & (pv[('count',  '3.0.8')]==0) & (pv[("count","3.0.9")]==0) & ((pv[("count","total")]>0))].sort_values("root_url")
bad_tracker_urls

count                     \
version                                            2.4.19 2.4.20 2.5.4 2.6.0   
root_url                                                                       
file:///C:/Users/Dell/Downloads/COVID-19%20Is%2...      0      4     0     0   
file://C:\Users\Dell\Downloads\COVID-19 Is Dist...      0      1     0     0   
file://F:\SINGAPORE JANICE\COURSES MODULES\9 Af...      0      1     0     0   
http://localhost:3000/zh-hk/credit-cards/coffee...      0      1     0     0   
http://webcache.googleusercontent.com/search            0      1     0     0   
https://blog.moneysmart.hk/zh-hk/credit-cards/%...      0      4     0     0   
https://blog3.moneysmart.sg/budgeting/fridge-re...      0      2     0     0   
https://blog3.moneysmart.sg/family/best-kids-sa...      0      5     0     0   
https://blog3.moneysmart.sg/healthcare/cpf-medi...      0     21     0     0   
https://blog3.moneysmart.sg/shopping/online-gro...      0      1     0     0   
https://get.moneysmart.sg/ocbc-cash-on-instalme...      0      2     0     0   
https://learn.moneysmart.sg/career/how-to-deter...      0      2     0     0   
https://learn.moneysmart.sg/cars/should-you-buy...      0      6     0     0   
https://learn.moneysmart.sg/investing/how-can-y...      0      1     0     0   
https://learn.moneysmart.sg/property/#home-loans        0      2     0     0   
https://learn.moneysmart.sg/property/should-you...      0      1     0     0   
https://staging.mssgdev.com/identity/signin             0      1     0     0   
https://staging.mssgdev.com/identity/signup             0      2     0     0   
https://webcache.googleusercontent.com/search           0      1     0     0   
https://www-new.moneysmart.hk/zh-hk/credit-card...      0      0     2     0   
https://www-new.moneysmart.hk/zh-hk/credit-card...      0      0     1     0   
https://www-new.moneysmart.sg/credit-cards              0      0     3     0   
https://www-new.moneysmart.sg/credit-cards/air-...      0      0     2     0   
https://www-new.moneysmart.sg/credit-cards/citi...      0      0    31     0   
https://www-new.moneysmart.sg/credit-cards/citi...      0      0     1     0   
https://www-new.moneysmart.sg/credit-cards/mayb...      0      0     1     0   
https://www-new.moneysmart.sg/credit-cards/stan...      0      0     1     0   
https://www-new.moneysmart.sg/credit-cards/stan...      0      0     1     0   
https://www-new.moneysmart.sg/personal-loan/scb...      0      0     1     0   
https://www.moneysmart.hk/zh-hk/credit-cards/ae...      0     14     0     0   
https://www.moneysmart.hk/zh-hk/credit-cards/am...      0      2     0     0   
https://www.moneysmart.hk/zh-hk/credit-cards/be...      0      3     0     0   
https://www.moneysmart.hk/zh-hk/credit-cards/bl...      0      1     0     0   
https://www.moneysmart.hk/zh-hk/credit-cards/ca...      0      2     0     0   
https://www.moneysmart.hk/zh-hk/credit-cards/ci...      0     10     0     0   
https://www.moneysmart.hk/zh-hk/credit-cards/ci...      0      2     0     0   
https://www.moneysmart.hk/zh-hk/credit-cards/ci...      0     17     0     0   
https://www.moneysmart.hk/zh-hk/credit-cards/da...      0      3     0     0   
https://www.moneysmart.hk/zh-hk/credit-cards/db...      0      6     0     0   
https://www.moneysmart.hk/zh-hk/credit-cards/db...      0      1     0     0   
https://www.moneysmart.hk/zh-hk/credit-cards/ea...      0     12     0     0   
https://www.moneysmart.hk/zh-hk/credit-cards/ha...      0      9     0     0   
https://www.moneysmart.hk/zh-hk/credit-cards/hs...      0      1     0     0   
https://www.moneysmart.hk/zh-hk/credit-cards/hs...      0      9     0     0   
https://www.moneysmart.hk/zh-hk/credit-cards/hs...      0      4     0     0   
https://www.moneysmart.hk/zh-hk/credit-cards/hs...      0      8     0     0   
https://www.moneysmart.hk/zh-hk/credit-cards/hs...      0     10     0     0   
https://www.moneysmart.hk/zh-hk/credit-cards/pl...  

In [161]:
non_ci_bad_tracker_urls = bad_tracker_urls[~bad_tracker_urls.index.str.contains("car-insurance")].sort_values(("count", "total"), ascending=False)
non_ci_bad_tracker_urls


count                     \
version                                            2.4.19 2.4.20 2.5.4 2.6.0   
root_url                                                                       
https://www.moneysmart.sg/identity/signin               0     50     0     0   
https://www-new.moneysmart.sg/credit-cards/citi...      0      0    31     0   
https://www.moneysmart.hk/zh-hk/credit-cards/st...      0     24     0     0   
https://blog3.moneysmart.sg/healthcare/cpf-medi...      0     21     0     0   
https://www.moneysmart.hk/zh-hk/credit-cards/ci...      0     17     0     0   
https://www.moneysmart.hk/zh-hk/credit-cards/we...      0     16     0     0   
https://www.moneysmart.hk/zh-hk/credit-cards/ae...      0     14     0     0   
https://www.moneysmart.hk/zh-hk/credit-cards/ea...      0     12     0     0   
https://www.moneysmart.sg/identity/signup               0     11     0     0   
https://www.moneysmart.sg/user-dashboard                0     10     0     0   
https://www.moneysmart.hk/zh-hk/credit-cards/ci...      0     10     0     0   
https://www.moneysmart.hk/zh-hk/credit-cards/hs...      0     10     0     0   
https://www.moneysmart.hk/zh-hk/credit-cards/hs...      0      9     0     0   
https://www.moneysmart.hk/zh-hk/credit-cards/ha...      0      9     0     0   
https://www.moneysmart.sg/travel-insurance/fwd-...      0      8     0     0   
https://www.moneysmart.hk/zh-hk/credit-cards/hs...      0      8     0     0   
https://learn.moneysmart.sg/cars/should-you-buy...      0      6     0     0   
https://www.moneysmart.hk/zh-hk/credit-cards/db...      0      6     0     0   
https://blog3.moneysmart.sg/family/best-kids-sa...      0      5     0     0   
file:///C:/Users/Dell/Downloads/COVID-19%20Is%2...      0      4     0     0   
https://www.moneysmart.hk/zh-hk/credit-cards/hs...      0      4     0     0   
https://www.moneysmart.sg/travel-insurance/fwd-...      0      4     0     0   
https://blog.moneysmart.hk/zh-hk/credit-cards/%...      0      4     0     0   
https://www.moneysmart.hk/zh-hk/credit-cards/da...      0      3     0     0   
https://www.moneysmart.sg/user-dashboard#               0      3     0     0   
https://www.moneysmart.hk/zh-hk/credit-cards/be...      0      3     0     0   
https://www.moneysmart.sg/user-dashboard/applic...      0      3     0     0   
https://www-new.moneysmart.sg/credit-cards              0      0     3     0   
https://learn.moneysmart.sg/property/#home-loans        0      2     0     0   
https://blog3.moneysmart.sg/budgeting/fridge-re...      0      2     0     0   
https://get.moneysmart.sg/ocbc-cash-on-instalme...      0      2     0     0   
https://www.moneysmart.sg/identity/email-verifi...      0      2     0     0   
https://learn.moneysmart.sg/career/how-to-deter...      0      2     0     0   
https://www.moneysmart.hk/zh-hk/credit-cards/st...      0      2     0     0   
https://www.moneysmart.hk/zh-hk/credit-cards/st...      0      2     0     0   
https://staging.mssgdev.com/identity/signup             0      2     0     0   
https://www-new.moneysmart.hk/zh-hk/credit-card...      0      0     2     0   
https://www.moneysmart.hk/zh-hk/credit-cards/ca...      0      2     0     0   
https://www-new.moneysmart.sg/credit-cards/air-...      0      0     2     0   
https://www.moneysmart.hk/zh-hk/credit-cards/ci...      0      2     0     0   
https://www.moneysmart.hk/zh-hk/credit-cards/am...      0      2     0     0   
https://www-new.moneysmart.hk/zh-hk/credit-card...      0      0     1     0   
https://www.moneysmart.sg/credit-cards/best-cre...      0      1     0     0   
file://F:\SINGAPORE JANICE\COURSES MODULES\9 Af...      0      1     0     0   
http://localhost:3000/zh-hk/credit-cards/coffee...      0      1     0     0   
https://www.moneysmart.sg/travel-insurance/hong...      0      1     0     0   
https://www.moneysmart.sg/travel-insurance/fwd-...      0      1     0     0   
http://webcache.googleusercontent.com/search        

In [162]:
bad_tracker_urls.to_csv("bad_tracker_urls.csv")

In [163]:
bad_tracker_urls.index.str.contains("car-insurance")

array([False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False])

In [164]:
from data_parsing import get_metadata_from_url

In [165]:
>> adding this from validating segment vs kinesis, but probably need to separate out into another library.


start_time = datetime.now()
print("starting at %s"%start_time.isoformat())
#This is a bit slow (consider at looking how to optimise, especially memory usage from creating loads of series objects
#Could probably optimise by splitting all the urls using a pandas function, then joining with a map to get page_type, path etc, but ymmv
metadata_df = merged_df.apply(lambda x: pd.Series(get_metadata_from_url(x.page_url)), axis=1)#, index=["page_type", "path", "ab_test", "country_code"])
end_time = datetime.now()
time_taken = (end_time-start_time).total_seconds()
print("Took %i seconds"%time_taken)

SyntaxError: invalid syntax (<ipython-input-165-2b699dbe46b7>, line 1)

In [ ]:
events.head()

In [ ]:
g_events = events[(events.page_url.str.contains("?"))] # 
                  #& (events.anonymous_id.isin(("8cce5789-b10f-4fda-934b-935b8356dd29", "af829a9c-fa98-4b60-b61b-f930e2cd5fcb")))]